# 🎬 Sistema de Recomendación con Filtrado Colaborativo (SVD)
**Autor:** Manuel Ortiz | **Curso:** Álgebra Lineal Aplicada a IA

Este proyecto utiliza la **Descomposición en Valores Singulares (SVD)** para implementar un sistema de recomendación por **Filtrado Colaborativo**. Buscamos patrones latentes de gustos para predecir ratings faltantes.

La matriz de ratings original $M$ se aproxima mediante el producto de tres matrices reducidas:

$$ M \approx M_k = U_k \Sigma_k V_k^T $$

Donde $k$ es el número de componentes latentes (características) que retenemos.


In [ ]:
import numpy as npimport pandas as pdfrom scipy.linalg import svdimport matplotlib.pyplot as plt
# Configuración de numpy para mostrar decimales de forma concisanp.set_printoptions(precision=4, suppress=True)

## 1. Definición de la Clase `SistemaRecomendacion`

In [ ]:
class SistemaRecomendacion:    """    Implementación de un sistema de recomendación basado en    Filtrado Colaborativo usando Descomposición en Valores Singulares (SVD)    """    def __init__(self, n_components=3):        self.n_components = n_components        self.U = None        self.sigma = None        self.Vt = None        self.matriz_predicciones = None    def entrenar(self, matriz_ratings):        print("\n--- ENTRENANDO MODELO ---")        n_usuarios, n_items = matriz_ratings.shape        # Aplicar SVD        self.U, sigma_valores, self.Vt = svd(matriz_ratings, full_matrices=False)        self.sigma = np.diag(sigma_valores)        # Reducción de dimensionalidad (Truncated SVD)        U_reducida = self.U[:, :self.n_components]        Sigma_reducida = self.sigma[:self.n_components, :self.n_components]        Vt_reducida = self.Vt[:self.n_components, :]        # Reconstrucción de la matriz de predicciones        self.matriz_predicciones = U_reducida @ Sigma_reducida @ Vt_reducida        # Calcular información retenida para mostrar        var_total = np.sum(sigma_valores**2)        var_retenida = np.sum(sigma_valores[:self.n_components]**2)        print(f"Dimensiones originales: {n_usuarios}x{n_items}")        print(f"Componentes latentes (k): {self.n_components}")        print(f"Información retenida: {100 * var_retenida / var_total:.2f}%")        print("---------------------------")    def predecir_rating(self, usuario_id, item_id):        if self.matriz_predicciones is None:            raise ValueError("Modelo no entrenado")        # Limitar la predicción al rango de ratings (0 a 5)        return np.clip(self.matriz_predicciones[usuario_id, item_id], 0, 5)    def recomendar_top_n(self, usuario_id, n=5, items_ya_vistos=None):        if self.matriz_predicciones is None:            raise ValueError("Modelo no entrenado")        predicciones = self.matriz_predicciones[usuario_id, :].copy()        if items_ya_vistos:            for item in items_ya_vistos:                # Calificación muy baja para no recomendar items ya vistos                predicciones[item] = -999        top_indices = np.argsort(predicciones)[::-1][:n]        # Devolver índice y rating predicho limitado        return [(idx, np.clip(predicciones[idx], 0, 5)) for idx in top_indices]    def evaluar_modelo(self, matriz_original):        mascara = matriz_original != 0        diferencias = matriz_original[mascara] - self.matriz_predicciones[mascara]        rmse = np.sqrt(np.mean(diferencias**2))        mae = np.mean(np.abs(diferencias))        return {'RMSE': rmse, 'MAE': mae}

## 2. Datos de Ejemplo y Entrenamiento

Matriz de 5 usuarios $\times$ 6 películas. El **0** indica una película no vista.

In [ ]:
# Matriz de ratings: Usuarios x Películas (0 = no visto)matriz_ratings = np.array([    [5, 3, 0, 1, 0, 4],  # Ana    [4, 0, 0, 1, 2, 3],  # Carlos    [1, 1, 0, 5, 4, 0],  # Diana    [1, 0, 0, 4, 0, 2],  # Eduardo    [0, 1, 5, 4, 0, 5]   # Fernanda])nombres_usuarios = ["Ana", "Carlos", "Diana", "Eduardo", "Fernanda"]nombres_peliculas = ["Inception", "Matrix", "Interstellar", "Terminator", "Avatar", "Blade Runner"]print("### Matriz Original ###")df_ratings = pd.DataFrame(matriz_ratings, index=nombres_usuarios, columns=nombres_peliculas)display(df_ratings)# Entrenar el modelo con k=3 componentes latentessistema = SistemaRecomendacion(n_components=3)sistema.entrenar(matriz_ratings)

## 3. Matriz Reconstruida y Predicciones

In [ ]:
print("### Matriz de Predicciones (Ratings Reconstruidos) ###")df_predicciones = pd.DataFrame(    np.round(sistema.matriz_predicciones, 2),    index=nombres_usuarios,    columns=nombres_peliculas)display(df_predicciones)

## 4. Generación de Recomendaciones Top-3

In [ ]:
print("\n--- Recomendaciones Top 3 por Usuario ---")print("""Nota: Solo se recomiendan películas NO vistas originalmente, "",      "basándose en la predicción del rating reconstruido."""
)for i, usuario in enumerate(nombres_usuarios):    # Identificar películas que el usuario YA ha visto    items_vistos = [j for j, r in enumerate(matriz_ratings[i]) if r > 0]        # Obtener el Top 3 de recomendaciones    recomendaciones = sistema.recomendar_top_n(i, n=3, items_ya_vistos=items_vistos)        print(f"⭐ Recomendaciones para **{usuario}**:")    for rank, (item_idx, rating) in enumerate(recomendaciones, 1):        print(f"  {rank}. {nombres_peliculas[item_idx]} (Predicción: {rating:.2f})")    print("")

## 5. Evaluación del Modelo

In [ ]:
metricas = sistema.evaluar_modelo(matriz_ratings)
print("### Métricas de Rendimiento (Evaluado solo en Ratings conocidos) ###")print(f"* **RMSE (Root Mean Square Error):** {metricas['RMSE']:.4f}")print(f"* **MAE (Mean Absolute Error):** {metricas['MAE']:.4f}")